# バッチエンドポイントへのデプロイ

ある診療所が一日中患者の測定を行い、各患者の詳細を個別のファイルに保存しているとする。そして一晩中、糖尿病予測モデルを使用して、その日のすべての患者データをバッチ処理し、翌朝に待機する予測を生成して、クリニックが糖尿病のリスクがあると予測される患者をフォローアップできるようにします。Azure Machine Learningでは、バッチエンドポイントを作成することでこれを実現できます。

## 始める前に

このノートブックのコードを実行するには、最新バージョンの **azure-ai-ml** パッケージが必要です。以下のセルを実行して、インストールされていることを確認してください。

> 注**：
> もし **azure-ai-ml** パッケージがインストールされていない場合は、`pip install azure-ai-ml` を実行してインストールしてください。

In [1]:
pip show azure-ai-ml

## ワークスペースに接続する

必要な SDK パッケージがインストールされ、ワークスペースに接続する準備が整いました。

ワークスペースに接続するには、サブスクリプションID、リソースグループ名、ワークスペース名といった識別子のパラメータが必要だ。Azure Machine Learningが管理するコンピュートインスタンスで作業しているので、デフォルト値を使用してワークスペースに接続できる。

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

## モデルの登録

バッチデプロイはワークスペースに登録されたモデルのみをデプロイすることができます。ローカルの `model` フォルダに格納されている MLflow モデルを登録します。

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

model_name = 'diabetes-mlflow'
model = ml_client.models.create_or_update(
    Model(name=model_name, path='./model', type=AssetTypes.MLFLOW_MODEL)
)

## バッチエンドポイントの作成

バッチエンドポイントは、アプリケーションがバッチスコアリングジョブをトリガーするために呼び出すことができる HTTPS エンドポイントです。バッチエンドポイント名は Azure リージョン内で一意である必要があります。datetime` 関数を使用して、現在の日付と時刻に基づいて一意の名前を生成します。

In [ ]:
import datetime

endpoint_name = "batch-" + datetime.datetime.now().strftime("%m%d%H%M%f")
endpoint_name

BatchEndpoint` クラスでエンドポイントを作成するには、名前とオプションで説明を指定する必要があります。エンドポイントを作成したら、そのエンドポイントにモデルをデプロイします。

In [ ]:
from azure.ai.ml.entities import BatchEndpoint

# create a batch endpoint
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="A batch endpoint for classifying diabetes in patients",
)

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style=「color:red;font-size:120%;background-color:yellow;font-weight:bold」> 重要です！続行する前に、エンドポイントが作成されるまで待ちます！スタジオに緑色の通知が表示されるはずです。</p>

## デプロイメントの作成

デプロイメントとは、実際の推論を行うモデルをホストするために必要なリソースのセットです。BatchDeployment` クラスを使用してエンドポイント用のデプロイメントを作成します。

MLflow モデルをデプロイするので、スコアリングスクリプトや環境の定義は必要ありません。Azure Machine Learning が自動的にそれらのアセットを作成してくれます。model` フォルダ内の `MLmodel` ファイルは、モデルの期待される入力と出力を理解するために使用されます。

以下のパラメータでモデルをデプロイします：

- name`： 配置の名前。
- description`： 配置が何を表しているかをさらに明確にするためのオプションの説明。
- endpoint_name`： モデルをデプロイするエンドポイントの名前。
- モデル`： 登録されているモデルの名前。
- compute`: コンピュート： 予測値を生成するためにデプロイされたモデルを呼び出す際に使用するコンピュート。
- インスタンス数 `instance_count`： 予測生成に使用するコンピュートノードの数。
- max_concurrency_per_instance`: 並列スコアリングスクリプトの最大実行数： 計算ノードあたりのスコアリングスクリプトの最大並列実行数。
- mini_batch_size`： スコアリングスクリプトの実行ごとに渡されるファイルの数。
- 出力アクション`： それぞれの新しい予測値が新しい行として出力ファイルに追加される。
- 出力ファイル名`： 予測値が追加されるファイル。
- retry_settings`: ミニバッチが失敗した場合の設定： ミニバッチが失敗した場合の設定。
- logging_level`： ログの冗長性レベル。許可される値は `warning`、`info`、`debug` である。

以下のセルを実行すると、デプロイが設定され、作成される。

In [ ]:
from azure.ai.ml.entities import BatchDeployment, BatchRetrySettings
from azure.ai.ml.constants import BatchDeploymentOutputAction

deployment = BatchDeployment(
    name="classifier-diabetes-mlflow",
    description="A diabetes classifier",
    endpoint_name=endpoint.name,
    model=model,
    compute="aml-cluster",
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=2,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
)
ml_client.batch_deployments.begin_create_or_update(deployment)

<p style=「color:red;font-size:120%;background-color:yellow;font-weight:bold」> 重要です！デプロイが完了するまでお待ちください！スタジオに緑色の通知が表示されるはずです。</p>

1つのエンドポイントに複数のモデルをデプロイできます。デフォルトのデプロイメントを設定すると、バッチエンドポイントを呼び出すときにデフォルトで使用するモデルを指定できます。

In [ ]:
endpoint.defaults = {}

endpoint.defaults["deployment_name"] = deployment.name

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style=「color:red;font-size:120%;background-color:yellow;font-weight:bold」> 重要です！続行する前に、デフォルトの配置が設定されるまで待ちます！スタジオに緑色の通知が表示されるはずです。</p>

## バッチ予測用のデータを準備する

data`フォルダにラベル付けされていないデータのCSVファイルがあります。data` フォルダ内のファイルを指すデータアセットを作成し、バッチジョブの入力として使用します。

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

data_path = "./data"
dataset_name = "patient-data-unlabeled"

patient_dataset_unlabeled = Data(
    path=data_path,
    type=AssetTypes.URI_FOLDER,
    description="An unlabeled dataset for diabetes classification",
    name=dataset_name,
)
ml_client.data.create_or_update(patient_dataset_unlabeled)

In [ ]:
patient_dataset_unlabeled = ml_client.data.get(
    name="patient-data-unlabeled", label="latest"
)

## ジョブの投入

バッチエンドポイントにモデルをデプロイし、ラベル付けされていないデータ資産を手に入れたので、ラベル付けされていないデータに対して予測を生成するためにエンドポイントを呼び出す準備ができました。

まず、登録したデータ資産を参照して入力を定義します。次にエンドポイントを起動し、パイプラインジョブを送信します。ジョブのURLを使用して、スタジオでジョブを監視できます。ジョブには、予測値を取得するための（生成された）スコアリングスクリプトの実行を表す子ジョブが含まれます。

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

input = Input(type=AssetTypes.URI_FOLDER, path=patient_dataset_unlabeled.id)

In [ ]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name, 
    input=input)

ml_client.jobs.get(job.name)

## 結果の取得

バッチエンドポイントを呼び出したパイプラインジョブが完了すると、結果を見ることができます。すべての予測結果は、デフォルトのデータストアに格納されている `predictions.csv` ファイルに収集されます。以下のセルを実行することで、ファイルをダウンロードし、データを可視化することができる。

In [ ]:
ml_client.jobs.download(name=job.name, download_path=".", output_name="score")

In [ ]:
with open("predictions.csv", "r") as f:
    data = f.read()

In [ ]:
from ast import literal_eval
import pandas as pd

score = pd.DataFrame(
    literal_eval(data.replace("\n", ",")), columns=["file", "prediction"]
)
score